TalkingData AdTracking Fraud Detection Challenge Submission
================================

 A TalkingData, a maior plataforma de big data independente da China, cobre mais de 70% dos dispositivos móveis ativos em todo o país. Eles lidam com 3 bilhões de cliques por dia, dos quais 90% são potencialmente fraudulentos. Sua abordagem atual para impedir fraudes de cliques para desenvolvedores de aplicativos é medir a jornada do clique de um usuário em todo o portfólio e sinalizar endereços IP que produzem muitos cliques, mas nunca acabam instalando aplicativos. Com essas informações, eles criaram uma lista negra de IP e uma lista negra de dispositivos.

  Embora bem-sucedidos, eles querem estar sempre um passo à frente dos fraudadores e pediram ajuda à comunidade de Kaggle para desenvolver ainda mais a solução. Em sua segunda competição com a Kaggle, você é desafiado a criar um algoritmo que prevê se um usuário fará o download de um aplicativo depois de clicar em um anúncio de aplicativo para dispositivos móveis. Para dar suporte à sua modelagem, eles forneceram um conjunto de dados generoso cobrindo aproximadamente 200 milhões de cliques em 4 dias!

Bibliotecas
-----------

In [1]:
import sklearn
import numpy as np 
import pandas as pd
import lightgbm as lgb

In [2]:
%matplotlib inline

Leitura dos dados
-------------------

In [ ]:
data = pd.read_csv('train.csv')
#print data
y = data['is_attributed']
X = data.drop(['is_attributed', 'click_time', 'attributed_time'], axis=1)

In [4]:
print data

           ip  app  device  os  channel           click_time attributed_time  \
0       87540   12       1  13      497  2017-11-07 09:30:38             NaN   
1      105560   25       1  17      259  2017-11-07 13:40:27             NaN   
2      101424   12       1  19      212  2017-11-07 18:05:24             NaN   
3       94584   13       1  13      477  2017-11-07 04:58:08             NaN   
4       68413   12       1   1      178  2017-11-09 09:00:09             NaN   
5       93663    3       1  17      115  2017-11-09 01:22:13             NaN   
6       17059    1       1  17      135  2017-11-09 01:17:58             NaN   
7      121505    9       1  25      442  2017-11-07 10:01:53             NaN   
8      192967    2       2  22      364  2017-11-08 09:35:17             NaN   
9      143636    3       1  19      135  2017-11-08 12:35:26             NaN   
10      73839    3       1  22      489  2017-11-08 08:14:37             NaN   
11      34812    3       1  13      489 

Resumo estatítico
--------------------

In [5]:

print data.describe()

                  ip           app         device             os  \
count  100000.000000  100000.00000  100000.000000  100000.000000   
mean    91255.879670      12.04788      21.771250      22.818280   
std     69835.553661      14.94150     259.667767      55.943136   
min         9.000000       1.00000       0.000000       0.000000   
25%     40552.000000       3.00000       1.000000      13.000000   
50%     79827.000000      12.00000       1.000000      18.000000   
75%    118252.000000      15.00000       1.000000      19.000000   
max    364757.000000     551.00000    3867.000000     866.000000   

             channel  is_attributed  
count  100000.000000  100000.000000  
mean      268.832460       0.002270  
std       129.724248       0.047591  
min         3.000000       0.000000  
25%       145.000000       0.000000  
50%       258.000000       0.000000  
75%       379.000000       0.000000  
max       498.000000       1.000000  


Validação Cruzada
------------------

In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Criação e treinamento do modelo
-----------------------------------

In [55]:
train_data=lgb.Dataset(X_train,label=y_train)

param = {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.05,'max_bin':200}

param['metric'] = ['auc', 'binary_logloss']

num_round=50

lgbm=lgb.train(param,train_data,num_round)

ypred2=lgbm.predict(X_test)

print ypred2

[0.04053626 0.04053626 0.04053626 ... 0.04053626 0.04053626 0.0403681 ]


In [66]:
for i in range(0,25000):
    if ypred2[i]>=0.5:       # setting threshold to .5
       ypred2[i]=1
    else:  
       ypred2[i]=0

[0. 1.]


TypeError: 'inplace' is an invalid keyword argument for this function

[0.03816263 0.03816381 0.03817622 0.03851524 0.03902354 0.03907208
 0.03911964 0.03919604 0.03920877 0.03928153 0.03974966 0.03985906
 0.04007963 0.0400828  0.04012744 0.04013797 0.04018018 0.0402371
 0.04026408 0.04027131 0.04027709 0.0402814  0.0402814  0.04029122
 0.04029226 0.04030346 0.04032833 0.04033378 0.04034508 0.0403484
 0.0403484  0.04036059 0.04036772 0.04036773 0.0403681  0.0403681
 0.04036814 0.04036814 0.04036815 0.04036856 0.04037902 0.04039922
 0.04042833 0.04053626 0.04055652 0.04056133 0.04057654 0.04058568
 0.04058575 0.04062939 0.0406931  0.04075221 0.04077258 0.04080188
 0.04080195 0.04100008 0.04126464 0.04130918 0.04151701 0.04165723
 0.0423879  0.04264078 0.04278154 0.04284354 0.04286491 0.04289065
 0.04289565 0.04309496 0.04311977 0.04322461 0.04378729 0.04403211
 0.04467214 0.04488585 0.04493558 0.04496451 0.04497795 0.04513067
 0.04513112 0.04519541 0.04520149 0.0459988  0.0462521  0.04698356
 0.04822832 0.04834215 0.04840127 0.04878916 0.04915594 0.0491803

In [22]:
gbm = lgb.LGBMRegressor(boosting_type='gbdt',
                        objective='binary',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)

In [23]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='auc',
early_stopping_rounds=5)

[1]	valid_0's auc: 0.85243
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.895509
[3]	valid_0's auc: 0.895481
[4]	valid_0's auc: 0.916691
[5]	valid_0's auc: 0.916652
[6]	valid_0's auc: 0.916744
[7]	valid_0's auc: 0.916472
[8]	valid_0's auc: 0.916448
[9]	valid_0's auc: 0.945792
[10]	valid_0's auc: 0.945795
[11]	valid_0's auc: 0.945748
[12]	valid_0's auc: 0.945755
[13]	valid_0's auc: 0.94573
[14]	valid_0's auc: 0.943327
[15]	valid_0's auc: 0.943331
Early stopping, best iteration is:
[10]	valid_0's auc: 0.945795


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.05, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=20,
       n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

Prevendo classificação dos dados
-------------------------------------

In [9]:
#X_test = pd.read_csv('test.csv')

In [10]:
#print X_test.shape

In [24]:
#X_test.drop(['click_id', 'click_time'], axis=1, inplace=True)
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print y_pred

[0.30085356 0.30085356 0.30085356 ... 0.30085356 0.30085356 0.30095158]


In [25]:
len(y_pred)

25000

Acurácia das classificações feitas
---------------------------------------

In [27]:
print np.unique(y_pred)

[0.29985473 0.30021303 0.30065489 0.30066668 0.30068033 0.30085356
 0.30095158 0.30120276 0.30121982 0.30131798 0.30132136 0.30133843
 0.30142497 0.3016587  0.30222942 0.30255334 0.30303318 0.30304501
 0.3038067  0.303913   0.30453333 0.30473116 0.30550019 0.30682899
 0.30888704 0.30970065 0.30971263 0.30971678 0.31048361 0.31480209
 0.31618606 0.31649541 0.31711749 0.31843673 0.32056581 0.32223252
 0.32235559 0.32252054 0.32336552 0.32359551 0.32415965 0.32701377
 0.32851196 0.32988697 0.33291083 0.33323753 0.33514331 0.33751648
 0.33854472 0.34001473 0.34031597 0.34123572 0.34227708 0.3429218
 0.34373788 0.34454975 0.34482227 0.34578418 0.34662087 0.34923685
 0.3504795  0.35246649 0.35259808 0.35594959 0.35607368 0.3563588
 0.35820252 0.35933856 0.36277333 0.36466103 0.37003893 0.37609766
 0.37615561 0.37629097 0.37711129 0.3844399  0.38457325 0.38814066
 0.38865687 0.39741428 0.39933489 0.40355751 0.4224288  0.42554307
 0.42807323 0.44207353 0.45038938 0.4594069  0.47165027 0.508393

Guardando resultados
------------------------

In [13]:

df = pd.DataFrame(data=pred, columns=[ 'is_attributed'])
df.index.name = "click_id"
df.is_attributed = df.is_attributed.astype(int)


NameError: name 'pred' is not defined

In [ ]:

# print(df.is_attributed.unique())
df.to_csv("saida.csv")


In [ ]:
print df.head()

In [ ]:
oi = pd.read_csv('saida.csv')

In [ ]:
print oi.shape

In [ ]:
print oi.head()